# Подготовка данных для обучения моделей

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* https://scikit-learn.org/stable/modules/compose.html#pipeline-chaining-estimators
* https://pytorch.org/docs/stable/data.html
* https://pytorch.org/tutorials/beginner/data_loading_tutorial.html
* Deep Learning with PyTorch (2020) Авторы: Eli Stevens, Luca Antiga, Thomas Viehmann


## Задачи для совместного разбора

1. Рассмотрите, как можно выстраивать конвейер предобработки данных при помощи `Pipeline` из `sklearn`

In [ ]:
from sklearn.datasets import make_regression
from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline

In [ ]:
X, y = make_regression(
    n_samples=1000,
    n_features=5
)

In [ ]:
pipe = Pipeline(
    [
        ("scaling", MinMaxScaler()),
        ("poly", PolynomialFeatures()),
        ("lr", LinearRegression()),
    ]
).fit(X, y)

In [ ]:
pipe.predict(X)

2. Создайте синтетический датасет для задачи регрессии и представьте его в виде `torch.utils.data.Dataset`

In [ ]:
pd.set_option('mode.chained_assignment', None)

In [ ]:
from torch.utils.data import Dataset, DataLoader

In [ ]:
a = [1, 2, 3]
a[2] # a.__getitem__(2)
len(a)

In [ ]:
class SyntDataset(Dataset):
  def __init__(self, transform: callable = None, **make_regression_args):
    self.X, self.y = make_regression(**make_regression_args)
    # self.X = th.Tensor(self.X)
    self.transform = transform

  def __getitem__(self, idx):
    # return {
    #     "x": self.X[idx],
    #     "y": self.y[idx]
    # }
    x = self.X[idx]
    if self.transform is not None:
      x = self.transform(x)
    return x, self.y[idx]

  def __len__(self):
    return len(self.X)

In [ ]:
import numpy as np
def add_squares(x):
  return np.c_[x, x ** 2]

In [ ]:
x = np.random.randint(0, 200, (20, 5))
x.shape

(20, 5)

In [ ]:
add_squares(x).shape

(40, 5)

In [ ]:
class SquareN:
  def __init__(self, n):
    self.n = n

  def __call__(self, x):
    for _ in range(self.n):
      x = add_squares(x)
    return x

In [ ]:
o = SquareN(2)
# o(d.X[:4])
# add_squares(d.X[:4])

In [ ]:
# d = SyntDataset(n_samples=1000, n_features=5)
d = SyntDataset(
    transform=SquareN(2),
    n_samples=1000,
    n_features=5
)
d[0:5]

In [ ]:
d[:5]

In [ ]:
dl = DataLoader(d, batch_size=32)

In [ ]:
for (batch_X, batch_y) in dl:
  print(batch_X.shape, batch_y.shape)
  break

torch.Size([32, 20]) torch.Size([32])


In [ ]:
def clip(x):
  return np.clip(x, 0, 1)

def add_squaresN(x, n):
  for _ in range(n):
    x = add_squares(x)
  return x

In [ ]:
x = np.random.randint(0, 200, (20, 5))
# x = clip(x)
# x = add_squaresN(x, 2)

for f in [clip, SquareN(2)]:
  x = f(x)

## Задачи для самостоятельного решения

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MaxAbsScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import torch
from torch.utils.data import Dataset

<p class="task" id="1"></p>

### 1
Считайте файл `bank-full.csv` ([источник](https://www.kaggle.com/datasets/hariharanpavan/bank-marketing-dataset-analysis-classification)) в виде `pd.DataFrame`. Используя `Pipeline` из `sklearn`, закодируйте значения в нечисловых столбцах целыми числами, после чего нормализуйте получившиеся признаки. Выведите преобразованные данные на экран.

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Deep Learning/data/bank-full.csv')
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [ ]:
cat = df.select_dtypes(include=['object']).columns.tolist()
num = df.select_dtypes(exclude=['object']).columns.tolist()

cat_pipeline = Pipeline(steps=[
    ('ohe', OneHotEncoder()),
    ('mmscl', MaxAbsScaler())])

num_pipeline = Pipeline(steps=[
    ('mmscl', MaxAbsScaler())])

pipeline = ColumnTransformer(
    transformers=[('num', num_pipeline, num),
                  ('cat', cat_pipeline, cat)])

df_encoded = pd.DataFrame(Pipeline(steps=[('preprocessor', pipeline)]).fit_transform(df))

df_encoded

,0,1,2,3,4,5,6,7,8,9,...,43,44,45,46,47,48,49,50,51,52
0,0.610526,0.020984,0.161290,0.053070,0.015873,-0.001148,0.000000,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
1,0.463158,0.000284,0.161290,0.030704,0.015873,-0.001148,0.000000,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
2,0.347368,0.000020,0.161290,0.015453,0.015873,-0.001148,0.000000,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
3,0.494737,0.014746,0.161290,0.018707,0.015873,-0.001148,0.000000,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
4,0.347368,0.000010,0.161290,0.040260,0.015873,-0.001148,0.000000,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,0.536842,0.008078,0.548387,0.198658,0.047619,-0.001148,0.000000,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
45207,0.747368,0.016930,0.548387,0.092721,0.031746,-0.001148,0.000000,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
45208,0.757895,0.055960,0.548387,0.229158,0.079365,0.211251,0.010909,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
45209,0.600000,0.006541,0.548387,0.103294,0.063492,-0.001148,0.000000,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0


<p class="task" id="2"></p>

### 2
Опишите класс `BankDatasetBase`. Решение должно удовлетворять следующим критериям:

* класс наследуется от `torch.utils.data.Dataset`;
* при создании объекта в конструктор передается набор данных в виде `pd.DataFrame`;
* объекты класса имеют поля `X` и `y` с признаками и метками соответственно;
* класс реализует интерфейс последовательностей (`__getitem__` + `__len__`);
* `obj[i]` возвращает кортеж, содержащий `i`-ую строку из `obj.X` и `i`-ую строку из `obj.y`.
    
Создайте объект класса `BankDatasetBase` и продемонстрируйте работоспособность.

In [ ]:
class BankDatasetBase(Dataset):
    def __init__(self, data: pd.DataFrame) -> None:
        self.data = data
        self.x = data.drop(columns=['y'])
        self.y = data['y']

    def __getitem__(self, idx):
        return self.x.iloc[idx, :].values, self.y.iloc[idx]

    def __len__(self):
        return len(self.data)

In [ ]:
bank_dataset = BankDatasetBase(df)
bank_dataset[0]

(array([58, 'management', 'married', 'tertiary', 'no', 2143, 'yes', 'no',
        'unknown', 5, 'may', 261, 1, -1, 0, 'unknown'], dtype=object),
 'no')

In [ ]:
len(bank_dataset)

45211

<p class="task" id="3"></p>

### 3
Опишите класс `BankDataset`. Решение должно удовлетворять всем критериям из предыдущего задания, а также:
* при создании объекта в конструктор может быть передан необязательный аргумент `transform: callable`;
* если данный аргумент был передан, то при получении `i`-го элемента, нужно вызвать `transform(x, y)` и вернуть полученный результат.

Создайте объект класса `BankDataset` и продемонстрируйте работоспособность.

In [ ]:
class BankDataset(Dataset):
    def __init__(self, data: pd.DataFrame, transform: callable = None) -> None:
        self.data = data
        self.x = data.drop(columns=['y'])
        self.y = data['y']
        self.transform = transform

    def __getitem__(self, idx) -> tuple:
        x = self.x
        y = self.y

        if self.transform is not None:
            x, y = self.transform(x, y)

        if isinstance(x, torch.Tensor):
            return x[idx], y[idx]

        return x.iloc[idx], y.iloc[idx]

    def __len__(self) -> int:
        return len(self.data)

def f(x, y):
    x = x.applymap(lambda x: x.upper() if isinstance(x, str) else x)
    return x, y

In [ ]:
bank_dataset1 = BankDataset(df, f)
bank_dataset1[42]

(age                   60
 job          BLUE-COLLAR
 marital          MARRIED
 education        UNKNOWN
 default               NO
 balance              104
 housing              YES
 loan                  NO
 contact          UNKNOWN
 day                    5
 month                MAY
 duration              22
 campaign               1
 pdays                 -1
 previous               0
 poutcome         UNKNOWN
 Name: 42, dtype: object,
 'no')

In [ ]:
bank_dataset2 = BankDataset(df)
bank_dataset2[42]

(age                   60
 job          blue-collar
 marital          married
 education        unknown
 default               no
 balance              104
 housing              yes
 loan                  no
 contact          unknown
 day                    5
 month                may
 duration              22
 campaign               1
 pdays                 -1
 previous               0
 poutcome         unknown
 Name: 42, dtype: object,
 'no')

In [ ]:
len(bank_dataset1), len(bank_dataset2)

(45211, 45211)

<p class="task" id="4"></p>

### 4
Опишите класс `LabelEncoderTransform`. Решение должно удовлетворять следующим критериям:

* при создании объекта в конструктор передаются названия нечисловых столбцов в датасете
* класс реализует интерфейс `Callable` (`__call__`); метод `__call__` имеет два параметра (признаки и класс) и возвращает кортеж, состоящих двух элементов: набора признаков, в котором нечисловые характеристики закодированы целыми числами; и чисел - меток классов;
* состояние объекта (индексы для кодирования) обновляется в момент очередного вызова `__call__` (т.е. все данные сразу никогда не передаются никакому методу объекта).
    
Продемонстрируйте работоспособность, создав объект `BankDataset` и передав при создании объект класса `LabelEncoderTransform`.

In [ ]:
class LabelEncoderTransform:
    def __init__(self, category_columns: list[str]) -> None:
        self.label_encoder = LabelEncoder()
        self.category_columns = category_columns

    def __call__(self, x, y) -> tuple:
        for col in self.category_columns:
            if col in x.columns:
                x[col] = self.label_encoder.fit_transform(x[col])
            else:
                y = pd.Series(self.label_encoder.fit_transform(y))
        return x, y

In [ ]:
category_columns = ['y', 'job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome']
bank_dataset3 = BankDataset(df, LabelEncoderTransform(category_columns))
bank_dataset3[42]

(age           60
 job            1
 marital        1
 education      3
 default        0
 balance      104
 housing        1
 loan           0
 contact        2
 day            5
 month          8
 duration      22
 campaign       1
 pdays         -1
 previous       0
 poutcome       3
 Name: 42, dtype: int64,
 0)

<p class="task" id="5"></p>

### 5
Опишите класс `ToTensor`.  Решение должно удовлетворять следующим критериям:
* класс реализует интерфейс `Callable` (`__call__`); метод `__call__` принимает имеет параметра (признаки и класс в числовом виде) и возвращает кортеж, состоящих 2 тензоров.

Опишите класс `Compose`.  Решение должно удовлетворять следующим критериям:
* при создании объекта в конструктор передается список объектов `transforms`, каждый из которых имеет метод `__call__(x, y)`;
* класс реализует интерфейс `Callable` (`__call__`); метод `__call__` принимает имеет параметра (признаки и класс в числовом виде) и и возвращает кортеж, полученный путем последовательного вызова объектов из `transforms`.

Продемонстрируйте работоспособность, создав объект `BankDataset` и передав при создании список из объектов LabelEncoderTransform и ToTensor.

In [ ]:
class ToTensor(object):
    def __call__(self, X, y) -> tuple:
        return torch.tensor(X.values), torch.tensor(y.values)

class Compose(object):
    def __init__(self, transforms: list) -> None:
        self.transforms = transforms

    def __call__(self, X, y):
        for transform in self.transforms:
            X, y = transform(X, y)
        return X, y

In [ ]:
bank_dataset4 = BankDataset(df, Compose([LabelEncoderTransform(category_columns), ToTensor()]))
bank_dataset4[42]

(tensor([ 60,   1,   1,   3,   0, 104,   1,   0,   2,   5,   8,  22,   1,  -1,
           0,   3]),
 tensor(0))

<p class="task" id="6"></p>

### 6
Разделите датасет из предыдущего задания на обучающую и тестовую выборку в соотношении 75% на 25%. Создайте объект `DataLoader` для получения пакетов размера 64, полученных из перемешанного обучающего датасета. Кастомизируйте `DataLoader` таким образом, чтобы пакет признаков был представлен в виде трехмерного тензора размера 64x2x8 (разделите 16 признаков на два тензора по 8). Получите один пакет и выведите на экран размерность тензоров пакета.

## Обратная связь
- [ ] Хочу получить обратную связь по решению